In [1]:
import altair as alt
import pandas as pd
import numpy as np
from scipy.stats import beta

In [88]:
z95 = 1.959963984540054
results = pd.read_csv('sem_results.csv', index_col=0)
method_names = results['methodName'].unique()
timeseries_lengths = results['timeSeriesLength'].unique()


#calculate which experiments had the true mean within the estimatimed_mean +/- abs(SEM)
###If using the frequentist CI below, calculate a rate instead of a count:
results['rate'] = results.apply(lambda row: float(np.abs(row['estMean']) < z95*row['SEM'])*0.01, axis=1)

#group by all the experimental conditions, sum the number of correct SEMs per condition,
#then flatten into long format with reset index:
data = pd.DataFrame(results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['rate'].sum()).reset_index()

#calculate confidence intervals for proportions:
##This uses standard binomial confidence interval:
##(equivalent to from statsmodels.stats.proportion import proportion_confint)
data['confInt'] = data.apply(lambda row: 1.96*np.sqrt(row['rate']*(1-row['rate'])/100), axis=1).fillna(0)

##Alternatively, can calculate the CI using a bayesian conjugate prior (beta distribution)
##I am using a beta(1,1) prior, hence the '+1's. Statsmodels has the Jeffreys interval which uses (0.5,0.5)
data['confIntLow'] = data.apply(lambda row: beta.ppf(0.025, 100*row['rate']+1, 100-100*row['rate']+1), axis=1)
data['confIntHigh'] = data.apply(lambda row: beta.ppf(0.975, 100*row['rate']+1, 100-100*row['rate']+1), axis=1)

In [89]:
#This is simply to avoid formatting the faceted plot later:
data['trueRho'] =data.apply(lambda row: 'ρ='+str(row['trueRho']),axis=1)

In [90]:
#see how poorly the automated block averaging estimates SEM particularly for low rho cases:
#results.groupby(['methodName', 'trueRho', 'timeSeriesLength'])['SEM'].std().unstack()

#Turn SEM into an estimate of the correlation time:
#results


In [91]:
data

,methodName,timeSeriesLength,trueRho,rate,confInt,confIntLow,confIntHigh
0,AR1_Bayes,30,ρ=0.1,0.93,0.050009,0.862404,0.965183
1,AR1_Bayes,30,ρ=0.3,0.98,0.027440,0.930295,0.993832
2,AR1_Bayes,30,ρ=0.5,0.99,0.019502,0.946068,0.997593
3,AR1_Bayes,30,ρ=0.7,0.92,0.053173,0.849884,0.958441
4,AR1_Bayes,30,ρ=0.9,0.89,0.061326,0.813480,0.937077
...,...,...,...,...,...,...,...
120,Sokal,3000,ρ=0.1,0.96,0.038408,0.901695,0.983733
121,Sokal,3000,ρ=0.3,0.96,0.038408,0.901695,0.983733
122,Sokal,3000,ρ=0.5,0.94,0.046547,0.875174,0.971683
123,Sokal,3000,ρ=0.7,0.94,0.046547,0.875174,0.971683


In [99]:
c =np.array([-0.1, 2.5])

In [101]:
np.sign(c).sum()

0.0

# Static version:

In [92]:
# the base chart
base = alt.Chart(data).transform_calculate(
    x_jittered = '0.15*random()*datum.timeSeriesLength+datum.timeSeriesLength',
     #ymin="datum.rate-datum.confInt",
     #ymax="datum.rate+datum.confInt",
    ymin = "datum.confIntLow",
    ymax = "datum.confIntHigh",
    goal='0.95')



#generate the scatter points:
points = base.mark_point(filled=True).encode(
    x=alt.X('x_jittered:Q', scale=alt.Scale(type='log'), title='Length of Timeseries'),
    y=alt.Y('rate:Q', scale=alt.Scale(domain=[0,1.04]), title='Rate of correct SEM'),
    size=alt.value(80),
    color=alt.Color('methodName', legend=alt.Legend(title="SEM method")))

#generate the scatter points:
line = base.mark_line().encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('rate:Q'),
    color='methodName')

# scale=alt.Scale(type='log', domain=[0.001, 1000]),
#           axis=alt.Axis(tickCount=5)
    
#generate the 95% mark:
rule = base.mark_rule(color='black').encode(
    alt.Y('goal:Q'))

# generate the error bars:
# errorbars = base.mark_errorbar().encode(
#     alt.X("x_jittered:Q"),
#     alt.Y("ymin:Q", title=''),
#     alt.Y2("ymax:Q"),
#     color='methodName')

errorbars = base.mark_rule(strokeWidth=3).encode(
    alt.X("x_jittered:Q"),
    alt.Y("ymin:Q", title=''),
    alt.Y2("ymax:Q"),
    color='methodName'
)


selection = alt.selection_single(fields=['symbol']);

chart = alt.layer(
    
  errorbars,
  points,
  line,
  rule,
    
).properties(
    width=250,
    height=200
    ).add_selection(selection).facet(facet=alt.Facet('trueRho:N', 
                                                     title='Autocorrelation parameter (ρ)'), columns=3)


chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


chart

alt.FacetChart(...)

# Interactive version:


In [93]:


# the base chart
base = alt.Chart(data).transform_calculate(
    x_jittered = '0.15*random()*datum.timeSeriesLength+datum.timeSeriesLength',
    #ymin="datum.rate-datum.confInt",
    #ymax="datum.rate+datum.confInt",
    ymin = "datum.confIntLow",
    ymax = "datum.confIntHigh",
    goal='0.95')


selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )

#generate the scatter points:
points = base.mark_point(filled=True).add_selection(selector).encode(
    x=alt.X('x_jittered:Q', scale=alt.Scale(type='log'), title='Length of Timeseries'),
    y=alt.Y('rate:Q', scale=alt.Scale(domain=[0,1.04]), title='Rate of correct SEM'),
    size=alt.value(80),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey'),legend=alt.Legend(title='SEM Method-Click to highlight!')),
    tooltip=['methodName:N'],)
    #color='methodName')
    
    
# alt.Chart(iris).mark_point().encode(
#     x='petalWidth',
#     y='petalLength',    
#     color=alt.condition(click,
#                         'species:N', alt.value('lightgray'), 
#                         scale=palette,
#                         legend=None)
    
selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )
    
#generate the lines:
line = base.mark_line().add_selection(selector).encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('rate:Q'),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey')))
    #color='methodName')

# #generate the 95% mark:
# rule = base.mark_rule(color='black').encode(
#     alt.Y('goal:Q'))

selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )

errorbars = base.mark_rule(strokeWidth=3).add_selection(selector).encode(
    alt.X("x_jittered:Q"),
    alt.Y("ymin:Q", title=''),
    alt.Y2("ymax:Q"),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey')))
    #color='methodName')

In [94]:
chart = alt.layer(
    
  errorbars,
  points,
  line,
  rule,
    
).properties(
    width=250,
    height=200
).facet(facet=alt.Facet('trueRho:N', title='Autocorrelation parameter (ρ)'), columns=3)


In [95]:
chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


In [96]:
chart.interactive()

alt.FacetChart(...)